In [72]:
import os
import sys
import pandas as pd
import subprocess

In [4]:
input_matrix = '/home/laura/ANALYSIS/VARIANT_CALLING/COVID/Compare/test_recalibrate/compare/compare.tsv'
bam_folder = '/home/laura/ANALYSIS/VARIANT_CALLING/COVID/Compare/test_recalibrate/Bam'

In [108]:
df_matrix = pd.read_csv(input_matrix, sep="\t")

In [109]:
df_matrix.tail()

,Position,N,Samples,20273789,20306625,20273588,20279180BIS,20289895,20277245,20279180,20286419,20261163,20297088
8,MN908947.3|C|6286|T,10,"20273789,20306625,20273588,20279180BIS,2028989...",1,1,1,1,1,1,1,1,1,1
9,MN908947.3|G|21255|C,5,"20279180BIS,20277245,20279180,20261163,20297088",0,0,0,1,0,1,1,0,1,1
10,MN908947.3|G|29645|T,10,"20273789,20306625,20273588,20279180BIS,2028989...",1,1,1,1,1,1,1,1,1,1
11,MN908947.3|T|29623|C,1,20277245,0,0,0,0,0,1,0,0,0,0
12,MN908947.3|T|445|C,8,"20273789,20306625,20273588,20289895,20277245,2...",1,1,1,0,1,1,0,1,1,1


In [111]:
recalibrate_ddbb_vcf(df_matrix, bam_folder)

SAMPLE: 20273789 has been corrected in position 21255: 0=>1
SAMPLE: 20306625 has been corrected in position 21255: 0=>1
SAMPLE: 20273588 has been corrected in position 21255: 0=>1
SAMPLE: 20289895 has been corrected in position 21255: 0=>1
SAMPLE: 20286419 has been corrected in position 21255: 0=>1
[1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
[0, 0, 0, 0, 0, 1, 0, 0, 0, 0]
SAMPLE: 20279180 has been corrected in position 445: 0=>1
[1, 1, 1, '!', 1, 1, 1, 1, 1, 1]


,Position,N,Samples,20273789,20306625,20273588,20279180BIS,20289895,20277245,20279180,20286419,20261163,20297088
0,MN908947.3|A|23403|G,10,"20273789,20306625,20273588,20279180BIS,2028989...",1,1,1,1,1,1,1,1,1,1
1,MN908947.3|C|14408|T,10,"20273789,20306625,20273588,20279180BIS,2028989...",1,1,1,1,1,1,1,1,1,1
2,MN908947.3|C|22227|T,10,"20273789,20306625,20273588,20279180BIS,2028989...",1,1,1,1,1,1,1,1,1,1
3,MN908947.3|C|241|T,10,"20273789,20306625,20273588,20279180BIS,2028989...",1,1,1,1,1,1,1,1,1,1
4,MN908947.3|C|26801|G,10,"20273789,20306625,20273588,20279180BIS,2028989...",1,1,1,1,1,1,1,1,1,1
5,MN908947.3|C|27944|T,10,"20273789,20306625,20273588,20279180BIS,2028989...",1,1,1,1,1,1,1,1,1,1
6,MN908947.3|C|28932|T,10,"20273789,20306625,20273588,20279180BIS,2028989...",1,1,1,1,1,1,1,1,1,1
7,MN908947.3|C|3037|T,10,"20273789,20306625,20273588,20279180BIS,2028989...",1,1,1,1,1,1,1,1,1,1
8,MN908947.3|C|6286|T,10,"20273789,20306625,20273588,20279180BIS,2028989...",1,1,1,1,1,1,1,1,1,1
9,MN908947.3|G|21255|C,10,"20279180BIS,20277245,20279180,20261163,20297088",1,1,1,1,1,1,1,1,1,1


In [110]:
def recalibrate_ddbb_vcf(snp_matrix_ddbb, bam_folder):
    
    df_matrix = snp_matrix_ddbb
    
    sample_list_matrix = df_matrix.columns[3:]
    n_samples = len(sample_list_matrix)
    
    list_index_dropped = []
    #Iterate over non unanimous positions 
    for index, data_row in df_matrix[df_matrix.N < n_samples].iloc[:,3:].iterrows():
        #Extract its position
        whole_position = df_matrix.loc[index,"Position"]
        row_reference = whole_position.split('|')[0]
        row_position = int(whole_position.split('|')[2])
        row_alt_snp = whole_position.split('|')[3]

        #Use enumerate to retrieve column index (column ondex + 3)
        #find positions with frequency >80% in mpileup execution
        #Returns ! for coverage 0
        new_presence_row = [recheck_variant_mpileup(row_reference, row_position, row_alt_snp, df_matrix.columns[n + 3], x, bam_folder) for n,x in enumerate(data_row)]
        
        df_matrix.iloc[index, 3:] = new_presence_row
        df_matrix.loc[index, 'N'] = sum([x == 1 for x in new_presence_row])
        #logger.info(new_presence_row)

    return df_matrix

In [106]:
def recheck_variant_mpileup(reference_id, position, alt_snp, sample, previous_binary, bam_folder):
    """
    https://www.biostars.org/p/254287/
    In the pileup format (without -u or -g), each line represents a genomic position, consisting of chromosome name, 1-based coordinate, reference base, the number of reads covering the site, read bases, base qualities and alignment mapping qualities. Information on match, mismatch, indel, strand, mapping quality and start and end of a read are all encoded at the read base column. At this column, a dot stands for a match to the reference base on the forward strand, a comma for a match on the reverse strand, a '>' or '<' for a reference skip, ACGTN for a mismatch on the forward strand and acgtn for a mismatch on the reverse strand. A pattern \+[0-9]+[ACGTNacgtn]+ indicates there is an insertion between this reference position and the next reference position. The length of the insertion is given by the integer in the pattern, followed by the inserted sequence. Similarly, a pattern -[0-9]+[ACGTNacgtn]+ represents a deletion from the reference. The deleted bases will be presented as * in the following lines. Also at the read base column, a symbol ^ marks the start of a read. The ASCII of the character following ^ minus 33 gives the mapping quality. A symbol $ marks the end of a read segment
    """
    previous_binary = int(previous_binary)
    position = int(position)

    #Identify correct bam
    for root, _, files in os.walk(bam_folder):
        for name in files:
            filename = os.path.join(root, name)
            sample_file = name.split('.')[0]
            if name.startswith(sample) and sample_file == sample and name.endswith(".bam"):
                bam_file = filename
    #format position for mpileup execution (NC_000962.3:632455-632455)
    position_format = reference_id + ":" + str(position) + "-" + str(position)
    
    #Execute command and retrieve output
    cmd = ["samtools", "mpileup", "-aa", "-r", position_format, bam_file]
    #print(cmd)
    text_mpileup = subprocess.run(cmd,stdout=subprocess.PIPE, stderr=subprocess.PIPE, check=True, universal_newlines=True) 
    split_mpileup = text_mpileup.stdout.split()
    #Extract 5th column to find variants
    mpileup_reference = split_mpileup[0]
    mpileup_position = int(split_mpileup[1])
    mpileup_depth = split_mpileup[3]
    mpileup_variants = split_mpileup[4]
    variant_list = list(mpileup_variants)
    variant_upper_list = [x.upper() for x in variant_list]

    most_counted_variant = max(set(variant_upper_list), key = variant_upper_list.count)
    count_all_variants = {x:variant_upper_list.count(x) for x in variant_upper_list}
    freq_most_frequent = count_all_variants[most_counted_variant]/len(variant_upper_list)

    if mpileup_depth == '0':
        print('WARNING: SAMPLE: {} has 0 depth in position {}'.format(sample, position))
        return '!'
    if freq_most_frequent <= 0.8:
        print('SAMPLE: {} has heterozygous position at {} with frequency'.format(sample, position, freq_most_frequent))


    if reference_id != mpileup_reference:
        print('ERROR: References are different')
        sys.exit(1)
    else:
        if (most_counted_variant == ".") or (most_counted_variant == ",") or (most_counted_variant == "*") or (freq_most_frequent < 0.8) or (most_counted_variant != alt_snp):
            if previous_binary != 0:
                print('SAMPLE: {} has been corrected in position {}: {}=>0'.format(sample, position, previous_binary))
            return 0
        elif (most_counted_variant == alt_snp) and (freq_most_frequent >= 0.8):
            if previous_binary != 1:
                print('SAMPLE: {} has been corrected in position {}: {}=>1'.format(sample, position, previous_binary))
            return 1

In [70]:
print('SAMPLE: {} has 0 depth in position {}'.format('blabla', 250))

SAMPLE: blabla has 0 depth in position 250


In [63]:
recheck_variant_mpileup('MN908947.3', 14408, 'T', '20279180', '/home/laura/ANALYSIS/VARIANT_CALLING/COVID/Bam')

0.9895209580838323

In [24]:
alt_str='tt$tttttttttttTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTCTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTT$TTtT$TTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTT$TTTTtTTTTTTTTTTTTTttTTTTTTTTTTTTTTTTTTTTTTTtTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTT$TTTTTTTTTTTTTTtTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTttTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTT$TTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTtTTTTTTTTTTTTTTTTtTTTTTTTTTTTTTTTTTTTTTTTTTTTtTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTtTTTTTTTTTTTTTTTTTTTTTTTTTtTTTTTTTTTT'

In [29]:
alt_list = list(alt_str)

In [34]:
upper_list = [x.upper() for x in alt_list]

In [30]:
max(set(upper_list), key = upper_list.count)

'T'

In [55]:
count_all_variants = {x:upper_list.count(x) for x in upper_list}

In [56]:
count_all_variants

{'T': 661, '$': 6, 'C': 1}

In [57]:
count_all_variants['C']

1